In [1]:
# Example 1
# requirements
# pip install -U docarray==0.21.0
# link: https://medium.com/jina-ai/build-neural-text-search-engine-in-10-minutes-12d27c51dde1

########################################################################################
# Preload
from docarray import Document, DocumentArray
doc = Document(uri="https://www.gutenberg.org/files/1342/1342-0.txt").load_uri_to_text()
#                   ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^ - нужный документ
# doc = Document(uri="./untitled.txt").load_uri_to_text()

# break large text into smaller chunks
docs = DocumentArray(Document(text = s.strip()) for s in doc.text.split('\n') if s.strip())

# apply feature hashing to embed the DocumentArray
docs.apply(lambda doc: doc.embed_feature_hashing())

########################################################################################
# Search
def search(text:str, limit:int=5) -> list:
    query = Document(text=text).embed_feature_hashing().match(
        docs, limit=limit, exclude_self=True, metric="jaccard", use_scipy=True,
    )
    return query.matches[:, ('text', 'scores__jaccard')]

print(search("she entered the room"))

[['Darcy only, entered the room.', 'in the room.', 'She entered the room with an air more than usually ungracious, made no', 'which Elizabeth received from Jane as soon as she entered the room, and', 'husband, called out, as she entered the library,--'], [{'value': 0.5714285714285714}, {'value': 0.6666666666666666}, {'value': 0.6923076923076923}, {'value': 0.6923076923076923}, {'value': 0.7}]]


In [2]:
# Example 1
# requirements
# pip install torch
# pip install transformers
# pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cpu

In [3]:
import torch
from scipy.spatial import distance
from transformers import BertTokenizer, BertModel


########################################################################################
# Preload
# Загрузка токенизатора и модели RuBERT
tokenizer = BertTokenizer.from_pretrained("DeepPavlov/rubert-base-cased")
model = BertModel.from_pretrained("DeepPavlov/rubert-base-cased")

def calc_emb(text:str):
    # Токенизация текста
    inputs = tokenizer(text, return_tensors="pt")
    
    # Передача токенизированных входных данных модели RuBERT для получения эмбеддингов
    with torch.no_grad():
        outputs = model(**inputs)
    
    # Получение эмбеддингов для текста (выходы из слоя [CLS] в BERT)
    embeddings = outputs.last_hidden_state.mean(dim=1).squeeze()
    # print(embeddings.shape)
    return embeddings


db = {}
with open("./чай список.txt") as file:
    while line := file.readline():
        # print(line.rstrip())
        db[line] = calc_emb(line.rstrip())


# print(db)

########################################################################################
# Search
def search_bert(text:str, limit:int=5) -> list:
    result_dict = {}
    emb = calc_emb(text)
    for line in db:
        result_dict[line] = distance.cosine(calc_emb(line), emb)
    sorted_result_dict = sorted(result_dict.items(), key=lambda x:x[1], reverse=True)
    return sorted_result_dict[:limit]


print(search_bert("чай чёреный"))


Some weights of the model checkpoint at DeepPavlov/rubert-base-cased were not used when initializing BertModel: ['cls.predictions.bias', 'cls.predictions.decoder.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


[('чай травяной lipton гибискус шиповник\n', 0.7438883849640057), ('чай зелёный lipton шиповник имбирь женьшень\n', 0.7274236412416102), ('чай травяной greenfield земляника клюква\n', 0.7208609844883598), ('чай чёрный lipton липа мята\n', 0.7023058380911271), ('чай травяной lipton куркума имбирь\n', 0.7002760516544559)]


In [4]:
print(search_bert("чай чёреный пряный май"))

[('чай чёрный ahmad tea ассам\n', 0.5521512566293039), ('чай чёрный ahmad tea ягоды\n', 0.5480951210724654), ('чай зелёный mabroc цейлонский\n', 0.5411294881052207), ('чай зелёный ahmad tea\n', 0.5408244828390791), ('чай чёрный ahmad tea\n', 0.5391722501070467)]


In [5]:
print(search_bert("чай чёреный майский"))

[('чай чёрный lipton витамин c шиповник малина\n', 0.7166118357025655), ('чай зелёный lipton шиповник имбирь женьшень\n', 0.7034294149667552), ('чай травяной lipton гибискус шиповник\n', 0.6979015966827189), ('чай травяной greenfield земляника клюква\n', 0.6911029147631592), ('чай чёрный ahmad tea гибискус персик малина\n', 0.6755447546724337)]


In [6]:
%%timeit
print(search_bert("чай белый"))

[('чай травяной lipton гибискус шиповник\n', 0.6832863282156284), ('чай зелёный lipton шиповник имбирь женьшень\n', 0.6804893339603759), ('чай чёрный lipton витамин c шиповник малина\n', 0.6639946085073771), ('чай травяной milford эхинацея имбирь\n', 0.6614269657264884), ('чай травяной lipton облепиха\n', 0.6531111660180768)]
[('чай травяной lipton гибискус шиповник\n', 0.6832863282156284), ('чай зелёный lipton шиповник имбирь женьшень\n', 0.6804893339603759), ('чай чёрный lipton витамин c шиповник малина\n', 0.6639946085073771), ('чай травяной milford эхинацея имбирь\n', 0.6614269657264884), ('чай травяной lipton облепиха\n', 0.6531111660180768)]
[('чай травяной lipton гибискус шиповник\n', 0.6832863282156284), ('чай зелёный lipton шиповник имбирь женьшень\n', 0.6804893339603759), ('чай чёрный lipton витамин c шиповник малина\n', 0.6639946085073771), ('чай травяной milford эхинацея имбирь\n', 0.6614269657264884), ('чай травяной lipton облепиха\n', 0

In [7]:
########################################################################################
# Preload
# Загрузка токенизатора и модели RuBERT
tokenizer = BertTokenizer.from_pretrained("bert-base-multilingual-cased")
model = BertModel.from_pretrained("bert-base-multilingual-cased")

def calc_emb(text:str):
    # Токенизация текста
    inputs = tokenizer(text, return_tensors="pt")
    
    # Передача токенизированных входных данных модели RuBERT для получения эмбеддингов
    with torch.no_grad():
        outputs = model(**inputs)
    
    # Получение эмбеддингов для текста (выходы из слоя [CLS] в BERT)
    embeddings = outputs.last_hidden_state.mean(dim=1).squeeze()
    # print(embeddings.shape)
    return embeddings


db = {}
with open("./чай список.txt") as file:
    while line := file.readline():
        # print(line.rstrip())
        db[line] = calc_emb(line.rstrip())


# print(db)

########################################################################################
# Search
def search_bert(text:str, limit:int=5) -> list:
    result_dict = {}
    emb = calc_emb(text)
    for line in db:
        result_dict[line] = distance.cosine(calc_emb(line), emb)
    sorted_result_dict = sorted(result_dict.items(), key=lambda x:x[1], reverse=True)
    return sorted_result_dict[:limit]


print(search_bert("чай чёреный"))

tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/996k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/625 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/714M [00:00<?, ?B/s]

[('чай травяной ahmad tea грейпфрут гуарана\n', 0.46453326408237705), ('чай фруктовый milford клубника малина\n', 0.43117749814656414), ('чай травяной ahmad tea ромашка мёд лаванда\n', 0.428529712240173), ('чай зелёный tess имбирь фрукт дракона\n', 0.4157801148016389), ('чай зелёный ahmad tea без кофеина\n', 0.4134165701588288)]


In [8]:
print(search_bert("чай чёреный пряный май"))

[('чай травяной ahmad tea грейпфрут гуарана\n', 0.37894300182027396), ('чай зелёный ahmad tea без кофеина\n', 0.3578230607383074), ('чай чёрный ahmad tea кардамон\n', 0.353323167249037), ('чай чёрный lipton бергамот РУССКИЙ\n', 0.3477679632419224), ('чай травяной milford гуарана каркаде\n', 0.34192577688571724)]


In [9]:
print(search_bert("чай чёреный майский"))

[('чай зелёный ahmad tea без кофеина\n', 0.422605158936302), ('чай травяной ahmad tea грейпфрут гуарана\n', 0.3970274387581778), ('чай травяной ahmad tea ромашка мёд лаванда\n', 0.3887809239239096), ('чай фруктовый milford клубника малина\n', 0.3880194885633429), ('чай травяной milford 12 трав\n', 0.3818316031580169)]
